<a href="https://colab.research.google.com/github/seraffin/FailOmen/blob/master/Mariusz/DecisionTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.ensemble import ExtraTreesRegressor

# Helper libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time

from google.colab import drive

Using TensorFlow backend.


# Utilities

In [0]:
# Evaluation function

def evaluation(additionalPredictions, predictions, refYsupervisor, verbose = False):
  
  lenght = len(refYsupervisor)

  failPositions = [[] for y in range(lenght)]

  for i, a in enumerate(refYsupervisor):

    for j, b in enumerate(a):
      if b == 0 : failPositions[i].append(j);

  predictionsTemp = predictions.copy()
  predictionPositions = [[] for y in range(lenght)]


  for i, a in enumerate(predictionsTemp):

    if len(failPositions[i]) != 0:
      for j in range(1 + additionalPredictions):
        argmin = np.argmin(a)
        predictionPositions[i].append(argmin)
        predictionsTemp[i][argmin] = 1

  predictionHits = [[] for y in range(lenght)]

  for i, a in enumerate(failPositions):
    count = 0
    for j, b in enumerate(a):


      for c in predictionPositions[i]:
        if c == b : count += 1

    if len(failPositions) != 0:
      predictionHits[i] = count


  failsCount = 0
  hitsCount = 0
  for i, a in enumerate(refYsupervisor):
    j = 0

    for k, b in enumerate(a):
      if b < 1.0 : j += 1

  #  if j > 0 :
  #    print (i, '.', j, predictionHits[i])


    failsCount += j
    hitsCount += predictionHits[i]

  print('Percentage of fails predicted', hitsCount / failsCount * 100, '%')
  
  if verbose == True:
    for i in range(lenght):
      print(i ,len(failPositions[i]), len(predictionPositions[i]), predictionHits[i], sep='\t')

In [0]:
#Distribution of argmins through all the predictions
  
def argminsDistribution():
  
    i = 1
    tab = [0] * 542
    for a in predictions:
      j = 0
      for b in a:
        if b < 1.0 : j = j + 1

      #print (i, '. ', j, np.argmin(a))
      tab[np.argmin(a)] += 1
      i = i + 1

    i = 0
    for a in tab:
      if a > 0 : print ('position', i, '\targmin count', a)
      i += 1

In [0]:
def printFailDistributionStats(y):
  
  failCount = 0
  passCount = 0
  failBuildCount = 0

  for a in y:
    if a[np.argmin(a)] == 0 : failBuildCount += 1
    for b in a:
      if b == 0 : failCount += 1
      if b == 1 : passCount += 1
  print ('failBuildCount', failBuildCount)
  print ('passBuildCount', len(y) - failBuildCount)
  print ('failBuild share ', failBuildCount / len(y) * 100, '%', sep='')
  print ('failCount', failCount)
  print ('passCount', passCount)
  print ('fail share ', failCount / (failCount + passCount) * 100, '%', sep='')

In [0]:
#Distributions of argmins through all the predictions

def countArgminsDistribution(prediction):
  
  i = 1
  tab = [0] * 542
  for a in predictions:
    j = 0
    for b in a:
      if b < 1.0 : j = j + 1

    #print (i, '. ', j, np.argmin(a))
    tab[np.argmin(a)] += 1
    i = i + 1

  i = 0
  distributed_array = []
  for a in tab:
    if a > 0 : 
      print ('position', i, '\targmin count', a)
      distributed_array.append((i,a))
    i += 1
  create_plot(distributed_array)  
  return distributed_array

In [0]:
# Create a list of single member classes positions

def listSingleMemberClassesPositions(refY, printFlag):

  yList = refY.tolist()
  yDistinct = [ele for ind, ele in enumerate(yList) if ele not in yList[:ind]]

  classesPopulation = []
  singleMemberClassesPositions = []
  noSingleMemberClasses = 0

  for i, a in enumerate(yDistinct):
    population = 0
    memberPosition = 0
    for j, b in enumerate(yList):
      if a == b: 
        population += 1
        memberPosition = j

    classesPopulation.append(population)
  #  print(i, population)
    if population == 1: 
      noSingleMemberClasses += 1
      singleMemberClassesPositions.append(memberPosition)

  if printFlag == True: 
    print(noSingleMemberClasses)
  else:
    return singleMemberClassesPositions

In [0]:
def searchForFailed(yList, withpassed_data = False, share = 1):
  changedRowList = []
  not_add_passed_data = 0
  for row in range (len(yList)):
    if 0 in yList[row]:
      changedRowList.append(row)
      continue
    elif withpassed_data == True:
      if not not_add_passed_data % share:
        changedRowList.append(row)
    not_add_passed_data += 1
      
  return changedRowList

def returnFailedData(xList, yList, changedRowList):
  xFailed = []
  yFailed = []
  
  for row in changedRowList:
    xFailed.append(xList[row])
    yFailed.append(yList[row])
  xFailed = np.array(xFailed)
  yFailed = np.array(yFailed)
  return xFailed, yFailed

In [0]:
# Double the single member classes for stratified sampling

def appendToDataset(refSingleMemberClassesPositions, refDataset):
  
  newDataset = pd.DataFrame(columns=list(refDataset))
  for i, a in enumerate(refSingleMemberClassesPositions):
    newDataset = newDataset.append(refDataset.iloc[a, :])
  
  return refDataset.append(newDataset, ignore_index=True)

In [0]:
def checkGPUpresence():

  device_name = tf.test.gpu_device_name()
  if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
  print('Found GPU at: {}'.format(device_name))

# Prepare data

In [10]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#Importing dataset

dataset = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataToML_full6.csv', index_col=False)

In [0]:
# Split data set into parameters and labels

x = dataset.iloc[:, 0:23739].values
y = dataset.iloc[:, 23739:24281].values

In [0]:
#for i in np.nditer(x, op_flags=['readwrite']):
#  if i > 0: x[...] = 1

In [0]:
#for i, j in np.ndenumerate(x):
#  if j > 0: print(j)

In [0]:
#for i, j in np.nditer(x):
#  if x[i][j] > 0: x[i][j] = 1

In [16]:
printFailDistributionStats(y)

failBuildCount 1884
passBuildCount 8377
failBuild share 18.360783549361663%
failCount 3125
passCount 5558337
fail share 0.056190260762367876%


In [0]:
# Random sampling
'''
xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size = 0.2)

listSingleMemberClassesPositions(yTrain, True)
listSingleMemberClassesPositions(yTest, True)
'''

# Stratified sampling
'''
dataset = appendToDataset(listSingleMemberClassesPositions(y, False), dataset)

x = dataset.iloc[:, 0:23739].values
y = dataset.iloc[:, 23739:24281].values

from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=None)

for train_index, test_index in sss.split(x, y):
  xTrain, xTest = x[train_index], x[test_index]
  yTrain, yTest = y[train_index], y[test_index]
  
listSingleMemberClassesPositions(yTrain, True)
listSingleMemberClassesPositions(yTest, True)
'''

# Reduce overpopulated class with fully passed builds

xReduced, yReduced = returnFailedData(x, y, searchForFailed(y, withpassed_data=False, share = 1))
from sklearn.model_selection import train_test_split
xTrain, xTest, yTrain, yTest = train_test_split(xReduced, yReduced, test_size = 0.2, random_state=79)

# Prepare and train model

In [18]:
start_time = time.time()

model = ExtraTreesRegressor()
model.fit(xTrain, yTrain)

print("--- %s seconds ---" % (time.time() - start_time))

--- 771.3843276500702 seconds ---


# Evaluate

In [0]:
predictions = model.predict(xTest)

In [0]:
#Distributions of argmins through all the predictions
  
def argminsDistribution(): 
    i = 1
    tab = [0] * 542
    for a in predictions:
      j = 0
      for b in a:
        if b < 1.0 : j += 1

      #print (i, '. ', j, np.argmin(a))
      tab[np.argmin(a)] += 1
      i = i + 1

    i = 0
    for a in tab:
      if a > 0 : print ('position', i, '\targmin count', a)
      i += 1

In [0]:
#failsCount()

In [0]:
#argminsDistribution()

In [0]:
# Evaluate by probability - if the percentage parameter is set to 0.5 than 50% 
# of predictions will be taken into cosideration. When the strongest prediction 
# in given build is 0.9 then predictions from 0.9 to 0.95 will be 
# taken into consideration, if 0.8 than from 0.8 to 0.9

def evaluation2(percentage, predictions, refYsupervisor, verbose=False):
  
  lenght = len(refYsupervisor)
  failPositions = [[] for y in range(lenght)]
  failPredictions = predictions.copy()
  predictionPositions = [[] for y in range(lenght)]
  predictionHits = [0] * lenght
  validPredictionsCount = [0] * lenght

  for i, a in enumerate(refYsupervisor):

    for j, b in enumerate(a):
      if b == 0 : failPositions[i].append(j);
  
  for i, a in enumerate(predictions):
    probabilityLimit = np.amin(a) + ((1 - np.amin(a)) * percentage)

    for j, b in enumerate(a):
      if b <= probabilityLimit: 
        predictionPositions[i].append(j)
        validPredictionsCount[i] += 1
        
  for i, a in enumerate(failPositions):
    count = 0
    
    for b in a:

      for c in predictionPositions[i]:
        if c == b : count += 1

    if len(a) != 0:
      predictionHits[i] = count

  hitsCount = 0
  failsCount = 0
  
  for i in range(lenght):
   # print(predictionHits[i])
    hitsCount += predictionHits[i]
    failsCount += len(failPositions[i])  

  print('Percentage of fails predicted', hitsCount / failsCount * 100, '%',
        '//Average of valid predictions', sum(validPredictionsCount)/len(validPredictionsCount))
  
  if verbose == True:
    for i in range(lenght):
      print(i ,len(failPositions[i]), len(predictionPositions[i]), predictionHits[i], sep='\t')

In [24]:
for i in range(14):
  print(i + 1, ' ', end='')
  evaluation(i, predictions, yTest)

1  Percentage of fails predicted 4.675716440422323 %
2  Percentage of fails predicted 7.088989441930618 %
3  Percentage of fails predicted 9.200603318250378 %
4  Percentage of fails predicted 10.407239819004525 %
5  Percentage of fails predicted 11.010558069381599 %
6  Percentage of fails predicted 12.217194570135746 %
7  Percentage of fails predicted 12.82051282051282 %
8  Percentage of fails predicted 13.423831070889895 %
9  Percentage of fails predicted 14.027149321266968 %
10  Percentage of fails predicted 14.479638009049776 %
11  Percentage of fails predicted 14.781297134238311 %
12  Percentage of fails predicted 15.233785822021115 %
13  Percentage of fails predicted 15.987933634992457 %
14  Percentage of fails predicted 16.440422322775262 %


In [25]:
for i in range(21):
  print(round(0.05 * i, 3), ' ', end='')
  evaluation2(0.05 * i, predictions, yTest)

0.0  Percentage of fails predicted 7.088989441930618 % //Average of valid predictions 1.376657824933687
0.05  Percentage of fails predicted 7.088989441930618 % //Average of valid predictions 1.5119363395225465
0.1  Percentage of fails predicted 7.088989441930618 % //Average of valid predictions 1.5384615384615385
0.15  Percentage of fails predicted 7.088989441930618 % //Average of valid predictions 1.5543766578249336
0.2  Percentage of fails predicted 7.088989441930618 % //Average of valid predictions 1.5755968169761274
0.25  Percentage of fails predicted 8.446455505279035 % //Average of valid predictions 2.1246684350132625
0.3  Percentage of fails predicted 8.446455505279035 % //Average of valid predictions 2.151193633952255
0.35  Percentage of fails predicted 9.653092006033184 % //Average of valid predictions 2.713527851458886
0.4  Percentage of fails predicted 9.95475113122172 % //Average of valid predictions 2.726790450928382
0.45  Percentage of fails predicted 9.95475113122172 % /